In [ ]:
# Spark Session
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder.appName("Spark Introduction")
    # .config("spark.executor.instances", "2") # NORMALLY SPARK IGNORES THIS SETTING
    .config("spark.cores.max", "4")  # HARD SETTING
    .config("spark.executor.cores", "1")
    .config("spark.executor.memory", "1g")
    .master("spark://spark-master:7077")
    .getOrCreate()
)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/10/17 15:17:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
# DRIVER URL

spark.sparkContext._jsc.sc().uiWebUrl().get()

'http://35457ff6b52e:4040'

In [ ]:
# EXECUTORS INFO

import requests

app_id = spark.sparkContext.applicationId
r = requests.get(f"http://35457ff6b52e:4040/api/v1/applications/{app_id}/executors")

In [7]:
r.json()

[{'id': 'driver',
  'hostPort': '35457ff6b52e:45219',
  'isActive': True,
  'rddBlocks': 0,
  'memoryUsed': 0,
  'diskUsed': 0,
  'totalCores': 0,
  'maxTasks': 0,
  'activeTasks': 0,
  'failedTasks': 0,
  'completedTasks': 0,
  'totalTasks': 0,
  'totalDuration': 236152,
  'totalGCTime': 92,
  'totalInputBytes': 0,
  'totalShuffleRead': 0,
  'totalShuffleWrite': 0,
  'isBlacklisted': False,
  'maxMemory': 455501414,
  'addTime': '2024-10-15T17:07:02.606GMT',
  'executorLogs': {},
  'memoryMetrics': {'usedOnHeapStorageMemory': 0,
   'usedOffHeapStorageMemory': 0,
   'totalOnHeapStorageMemory': 455501414,
   'totalOffHeapStorageMemory': 0},
  'blacklistedInStages': [],
  'peakMemoryMetrics': {'JVMHeapMemory': 90823592,
   'JVMOffHeapMemory': 92036280,
   'OnHeapExecutionMemory': 0,
   'OffHeapExecutionMemory': 0,
   'OnHeapStorageMemory': 0,
   'OffHeapStorageMemory': 0,
   'OnHeapUnifiedMemory': 0,
   'OffHeapUnifiedMemory': 0,
   'DirectPoolMemory': 38186975,
   'MappedPoolMemory': 0,

In [ ]:
# Emp Data & Schema

emp_data = [
    ["001", "101", "John Doe", "30", "Male", "50000", "2015-01-01"],
    ["002", "101", "Jane Smith", "25", "Female", "45000", "2016-02-15"],
    ["003", "102", "Bob Brown", "35", "Male", "55000", "2014-05-01"],
    ["004", "102", "Alice Lee", "28", "Female", "48000", "2017-09-30"],
    ["005", "103", "Jack Chan", "40", "Male", "60000", "2013-04-01"],
    ["006", "103", "Jill Wong", "32", "Female", "52000", "2018-07-01"],
    ["007", "101", "James Johnson", "42", "Male", "70000", "2012-03-15"],
    ["008", "102", "Kate Kim", "29", "Female", "51000", "2019-10-01"],
    ["009", "103", "Tom Tan", "33", "Male", "58000", "2016-06-01"],
    ["010", "104", "Lisa Lee", "27", "Female", "47000", "2018-08-01"],
    ["011", "104", "David Park", "38", "Male", "65000", "2015-11-01"],
    ["012", "105", "Susan Chen", "31", "Female", "54000", "2017-02-15"],
    ["013", "106", "Brian Kim", "45", "Male", "75000", "2011-07-01"],
    ["014", "107", "Emily Lee", "26", "Female", "46000", "2019-01-01"],
    ["015", "106", "Michael Lee", "37", "Male", "63000", "2014-09-30"],
    ["016", "107", "Kelly Zhang", "30", "Female", "49000", "2018-04-01"],
    ["017", "105", "George Wang", "34", "Male", "57000", "2016-03-15"],
    ["018", "104", "Nancy Liu", "29", "Female", "50000", "2017-06-01"],
    ["019", "103", "Steven Chen", "36", "Male", "62000", "2015-08-01"],
    ["020", "102", "Grace Kim", "32", "Female", "53000", "2018-11-01"],
]

emp_schema = "employee_id string, department_id string, name string, age string, gender string, salary string, hire_date string"

In [3]:
# Create emp DataFrame

emp = spark.createDataFrame(data=emp_data, schema=emp_schema)

In [7]:
emp

DataFrame[employee_id: string, department_id: string, name: string, age: string, gender: string, salary: string, hire_date: string]

In [11]:
emp.show(1)

+-----------+-------------+--------+---+------+------+----------+
|employee_id|department_id|    name|age|gender|salary| hire_date|
+-----------+-------------+--------+---+------+------+----------+
|        001|          101|John Doe| 30|  Male| 50000|2015-01-01|
+-----------+-------------+--------+---+------+------+----------+
only showing top 1 row



In [4]:
# Check number of partitions

emp.rdd.getNumPartitions()

4

In [5]:
from pyspark.sql import functions as F

emp = emp.withColumn("partition_id", F.spark_partition_id())
emp.show()

+-----------+-------------+-------------+---+------+------+----------+------------+
|employee_id|department_id|         name|age|gender|salary| hire_date|partition_id|
+-----------+-------------+-------------+---+------+------+----------+------------+
|        001|          101|     John Doe| 30|  Male| 50000|2015-01-01|           0|
|        002|          101|   Jane Smith| 25|Female| 45000|2016-02-15|           0|
|        003|          102|    Bob Brown| 35|  Male| 55000|2014-05-01|           0|
|        004|          102|    Alice Lee| 28|Female| 48000|2017-09-30|           0|
|        005|          103|    Jack Chan| 40|  Male| 60000|2013-04-01|           0|
|        006|          103|    Jill Wong| 32|Female| 52000|2018-07-01|           1|
|        007|          101|James Johnson| 42|  Male| 70000|2012-03-15|           1|
|        008|          102|     Kate Kim| 29|Female| 51000|2019-10-01|           1|
|        009|          103|      Tom Tan| 33|  Male| 58000|2016-06-01|      

DATAFRAME OF PARTITIONS BY EXECUTOR

In [ ]:
import os
import re
import socket


def get_executor_partition_info(index, iterator):
    executor_log_url = os.getenv("SPARK_LOG_URL_STDERR")
    pattern = r"executorId=(\d+)"
    executor_id = re.findall(pattern, executor_log_url)[0]
    worker = socket.gethostname()

    # Return a string containing executor and partition information
    return [[index, worker, executor_id]]


# Apply the function and collect the results
info = emp.rdd.mapPartitionsWithIndex(get_executor_partition_info).toDF(
    ["partition_id", "worker_host", "executor_id"]
)
info.show()

+------------+------------+-----------+
|partition_id| worker_host|executor_id|
+------------+------------+-----------+
|           0|26130b90b9ae|          3|
|           1|26130b90b9ae|          2|
|           2|aa1460ac4ddd|          0|
|           3|aa1460ac4ddd|          1|
+------------+------------+-----------+



DATAFRAME WITH EXECUTOR AND PARTITION BY ROW

In [ ]:
import os
import re
import socket


def get_executor_partition_info(index, iterator):
    executor_log_url = os.getenv("SPARK_LOG_URL_STDERR")
    pattern = r"executorId=(\d+)"
    executor_id = re.findall(pattern, executor_log_url)[0]
    worker = socket.gethostname()

    return [
        [index, worker, executor_id] + list(row.asDict().values()) for row in iterator
    ]


emp2 = emp.rdd.mapPartitionsWithIndex(get_executor_partition_info).toDF(
    ["partition_id", "worker_host", "executor_id"] + emp.columns
)
emp2.show()

+------------+------------+-----------+-----------+-------------+-------------+---+------+------+----------+------------+
|partition_id| worker_host|executor_id|employee_id|department_id|         name|age|gender|salary| hire_date|partition_id|
+------------+------------+-----------+-----------+-------------+-------------+---+------+------+----------+------------+
|           0|aa1460ac4ddd|          0|        001|          101|     John Doe| 30|  Male| 50000|2015-01-01|           0|
|           0|aa1460ac4ddd|          0|        002|          101|   Jane Smith| 25|Female| 45000|2016-02-15|           0|
|           0|aa1460ac4ddd|          0|        003|          102|    Bob Brown| 35|  Male| 55000|2014-05-01|           0|
|           0|aa1460ac4ddd|          0|        004|          102|    Alice Lee| 28|Female| 48000|2017-09-30|           0|
|           0|aa1460ac4ddd|          0|        005|          103|    Jack Chan| 40|  Male| 60000|2013-04-01|           0|
|           1|26130b90b9

In [19]:
# Get stages in detail from rdd

print(emp.rdd.toDebugString().decode("utf8"))

(4) MapPartitionsRDD[23] at javaToPython at NativeMethodAccessorImpl.java:0 []
 |  MapPartitionsRDD[22] at javaToPython at NativeMethodAccessorImpl.java:0 []
 |  SQLExecutionRDD[21] at javaToPython at NativeMethodAccessorImpl.java:0 []
 |  MapPartitionsRDD[20] at javaToPython at NativeMethodAccessorImpl.java:0 []
 |  MapPartitionsRDD[4] at applySchemaToPythonRDD at NativeMethodAccessorImpl.java:0 []
 |  MapPartitionsRDD[3] at map at SerDeUtil.scala:69 []
 |  MapPartitionsRDD[2] at mapPartitions at SerDeUtil.scala:117 []
 |  PythonRDD[1] at RDD at PythonRDD.scala:53 []
 |  ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:287 []


In [20]:
# Write our first Transformation (EMP salary > 50000)
# THe thre forms are aliases. The physical plan is the same

emp_final1 = emp.where("salary > 50000")
emp_final2 = emp.filter("salary > 50000")
emp_final3 = emp.filter(F.col("salary") > 50000)

In [21]:
emp_final1.explain()

== Physical Plan ==
*(1) Filter (isnotnull(salary#5) AND (cast(salary#5 as int) > 50000))
+- *(1) Project [employee_id#0, department_id#1, name#2, age#3, gender#4, salary#5, hire_date#6, SPARK_PARTITION_ID() AS partition_id#84]
   +- *(1) Scan ExistingRDD[employee_id#0,department_id#1,name#2,age#3,gender#4,salary#5,hire_date#6]




In [22]:
emp_final2.explain()

== Physical Plan ==
*(1) Filter (isnotnull(salary#5) AND (cast(salary#5 as int) > 50000))
+- *(1) Project [employee_id#0, department_id#1, name#2, age#3, gender#4, salary#5, hire_date#6, SPARK_PARTITION_ID() AS partition_id#84]
   +- *(1) Scan ExistingRDD[employee_id#0,department_id#1,name#2,age#3,gender#4,salary#5,hire_date#6]




In [23]:
emp_final3.explain()

== Physical Plan ==
*(1) Filter (isnotnull(salary#5) AND (cast(salary#5 as int) > 50000))
+- *(1) Project [employee_id#0, department_id#1, name#2, age#3, gender#4, salary#5, hire_date#6, SPARK_PARTITION_ID() AS partition_id#84]
   +- *(1) Scan ExistingRDD[employee_id#0,department_id#1,name#2,age#3,gender#4,salary#5,hire_date#6]




In [25]:
emp_final1.count()

13

In [26]:
# Validate number of Partitions

emp_final1.rdd.getNumPartitions()

4

In [28]:
emp.filter(F.col("salary") < 49000).rdd.getNumPartitions()

4

In [31]:
emp_final1.count()

13

In [39]:
emp.printSchema()

root
 |-- employee_id: string (nullable = true)
 |-- department_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- age: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: string (nullable = true)
 |-- hire_date: string (nullable = true)
 |-- partition_id: integer (nullable = false)



In [11]:
emp.show()

+-----------+-------------+-------------+---+------+------+----------+------------+
|employee_id|department_id|         name|age|gender|salary| hire_date|partition_id|
+-----------+-------------+-------------+---+------+------+----------+------------+
|        001|          101|     John Doe| 30|  Male| 50000|2015-01-01|           0|
|        002|          101|   Jane Smith| 25|Female| 45000|2016-02-15|           0|
|        003|          102|    Bob Brown| 35|  Male| 55000|2014-05-01|           0|
|        004|          102|    Alice Lee| 28|Female| 48000|2017-09-30|           0|
|        005|          103|    Jack Chan| 40|  Male| 60000|2013-04-01|           0|
|        006|          103|    Jill Wong| 32|Female| 52000|2018-07-01|           1|
|        007|          101|James Johnson| 42|  Male| 70000|2012-03-15|           1|
|        008|          102|     Kate Kim| 29|Female| 51000|2019-10-01|           1|
|        009|          103|      Tom Tan| 33|  Male| 58000|2016-06-01|      

In [ ]:
# Write data as CSV output (ACTION)

emp.write.format("csv").mode("overwrite").option("header", True).save(
    "hdfs://namenode:9000/output/02/emp"
)

In [18]:
spark.read.option("header", True).csv("hdfs://namenode:9000/output/02/emp").show()

+-----------+-------------+-------------+---+------+------+----------+------------+
|employee_id|department_id|         name|age|gender|salary| hire_date|partition_id|
+-----------+-------------+-------------+---+------+------+----------+------------+
|        016|          107|  Kelly Zhang| 30|Female| 49000|2018-04-01|           3|
|        017|          105|  George Wang| 34|  Male| 57000|2016-03-15|           3|
|        018|          104|    Nancy Liu| 29|Female| 50000|2017-06-01|           3|
|        019|          103|  Steven Chen| 36|  Male| 62000|2015-08-01|           3|
|        020|          102|    Grace Kim| 32|Female| 53000|2018-11-01|           3|
|        011|          104|   David Park| 38|  Male| 65000|2015-11-01|           2|
|        012|          105|   Susan Chen| 31|Female| 54000|2017-02-15|           2|
|        013|          106|    Brian Kim| 45|  Male| 75000|2011-07-01|           2|
|        014|          107|    Emily Lee| 26|Female| 46000|2019-01-01|      

In [ ]:
spark.read.option("header", True).csv(
    "hdfs://namenode:9000/output/02/emp/part-00000-c13440d5-b4d0-4566-b606-0d9d5a554361-c000.csv"
).show()

+-----------+-------------+----------+---+------+------+----------+------------+
|employee_id|department_id|      name|age|gender|salary| hire_date|partition_id|
+-----------+-------------+----------+---+------+------+----------+------------+
|        001|          101|  John Doe| 30|  Male| 50000|2015-01-01|           0|
|        002|          101|Jane Smith| 25|Female| 45000|2016-02-15|           0|
|        003|          102| Bob Brown| 35|  Male| 55000|2014-05-01|           0|
|        004|          102| Alice Lee| 28|Female| 48000|2017-09-30|           0|
|        005|          103| Jack Chan| 40|  Male| 60000|2013-04-01|           0|
+-----------+-------------+----------+---+------+------+----------+------------+



In [19]:
spark.stop()